# IMDB電影評論資料探索分析

本筆記本進行IMDB資料集的探索性分析，包括：
- 資料載入和基本統計
- 文本長度分佈分析
- 詞彙統計和視覺化
- 情感標籤分佈
- 預處理效果比較

In [ ]:
# 導入必要套件
import sys
import os
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

# 自定義模組
from src.data import IMDBDataLoader, TextPreprocessor
from src.utils.logger import logger

# 設定圖表樣式
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("套件載入完成！")

In [ ]:
# 載入IMDB資料集
logger.info("開始載入IMDB資料集...")

data_loader = IMDBDataLoader(cache_dir="../data/raw")
train_texts, train_labels, test_texts, test_labels = data_loader.load_data()

logger.info(f"訓練集大小: {len(train_texts)}")
logger.info(f"測試集大小: {len(test_texts)}")

# 獲取資料統計信息
stats = data_loader.get_data_statistics()
for key, value in stats.items():
    print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

In [ ]:
# 預處理效果比較
sample_text = train_texts[0]
print("原始文本範例:")
print(sample_text[:500] + "...")
print("\n" + "="*50 + "\n")

# 基礎預處理
basic_preprocessor = TextPreprocessor(
    remove_html=True,
    remove_urls=True,
    lowercase=True,
    handle_negations=True
)

processed_text = basic_preprocessor.preprocess(sample_text)
print("基礎預處理後:")
print(processed_text[:500] + "...")